In [ ]:
import torch

In [1]:
# class torch.nn.Parameter():Tensor的一种，常被用于模块参数(module parameter)。
# Parameters 是 Variable 的子类。Paramenters和Modules一起使用的时候会有一些特殊的属性，
# 即：当Paramenters赋值给Module的属性的时候，他会自动的被加到 Module的 参数列表中(即：会出现在 parameters() 迭代器中)。
# 将Varibale赋值给Module属性则不会有这样的影响。
# Variable 与 Parameter的另一个不同之处在于，Parameter不能被 volatile(即：无法设置volatile=True)而且默认requires_grad=True。Variable默认requires_grad=False。
# 参数说明：
#     data (Tensor) – parameter tensor.
#     requires_grad (bool, optional) – 默认为True，在BP的过程中会对其求微分。

In [ ]:
# class torch.nn.Module：所有神经网络模型的基类。
# Modules也可以包含其它Modules,允许使用树结构嵌入他们。你可以将子模块赋值给模型属性。
import torch.nn as nn
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = nn.Conv2d(1,20,5) # 当前的nn.Conv2d模块就被赋值成为Model模块的一个子模块，成为“树结构”的叶子
        self.conv2 = nn.Conv2d(20,20,5)
    def forward(self,x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))
# 通过上面方式赋值的submodule会被注册。当调用 .cuda() 的时候，submodule的参数也会转换为cuda Tensor。

In [3]:
# add_module(name, module)：将一个 child module 添加到当前 modle。被添加的module可以通过 name属性来获取。
import torch.nn as nn
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.add_module('conv',nn.Conv2d(10,20,4))
        # self.conv = nn.Conv2d(10,20,4) 和上面这个增加module的方式等价
model = Model()

In [4]:
# children():返回当前模型 子模块的迭代器。
import torch.nn as nn
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.add_module('conv',nn.Conv2d(10,20,4))
        self.add_module('conv2',nn.Conv2d(20,10,4))
model = Model()
for sub_module in model.children():
    print(sub_module)

Conv2d(10, 20, kernel_size=(4, 4), stride=(1, 1))
Conv2d(20, 10, kernel_size=(4, 4), stride=(1, 1))


In [5]:
# modules():返回一个包含 当前模型 所有模块的迭代器。
#  重复的模块只被返回一次(children()也是)
import torch.nn as nn
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.add_module("conv", nn.Conv2d(10, 20, 4))
        self.add_module("conv1", nn.Conv2d(20 ,10, 4))
model = Model()
for module in model.modules():
    print(module)

Model(
  (conv): Conv2d(10, 20, kernel_size=(4, 4), stride=(1, 1))
  (conv1): Conv2d(20, 10, kernel_size=(4, 4), stride=(1, 1))
)
Conv2d(10, 20, kernel_size=(4, 4), stride=(1, 1))
Conv2d(20, 10, kernel_size=(4, 4), stride=(1, 1))


In [9]:
# named_children():返回 包含 模型当前子模块 的迭代器，yield 模块名字和模块本身。
for name,module in model.named_children():
    if name in ['conv','conv1']:
        print(module)

Conv2d(10, 20, kernel_size=(4, 4), stride=(1, 1))
Conv2d(20, 10, kernel_size=(4, 4), stride=(1, 1))


In [ ]:
# torch.nn.Sequential(* args)：一个时序容器。Modules 会以他们传入的顺序被添加到容器中。当然，也可以传入一个OrderedDict。
model = nn.Sequential(
    nn.Conv2d(1,20,5),
    nn.ReLU(),
    nn.Conv2d(20,64,5),
    nn.ReLU()
)
model = nn.Sequential(OrderedDict([
    ('conv1',nn.Conv2d(1,20,5)),
    ('relu1',nn.ReLU()),
    ('conv2',nn.Conv2d(20,64,5)),
    ('relu2',nn.ReLU())
]))

In [10]:
# torch.nn.ModuleList(modules=None)[source]:将submodules保存在一个list中。
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule,self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10,10) for i in range(10)])
    def forward(self,x):
        for i,l in enumerate(self.linears):
            x = self.linears[i//2](x) + l(x)
        return x

In [17]:
# append(module)[source]:等价于 list 的 append()
# extend(modules)[source]:等价于 list 的 extend() 方法
model = MyModule()
model.linears.append(nn.Conv2d(10,20,5))
for m in model.modules():
    print(m)

MyModule(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
    (5): Linear(in_features=10, out_features=10, bias=True)
    (6): Linear(in_features=10, out_features=10, bias=True)
    (7): Linear(in_features=10, out_features=10, bias=True)
    (8): Linear(in_features=10, out_features=10, bias=True)
    (9): Linear(in_features=10, out_features=10, bias=True)
    (10): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  )
)
ModuleList(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): Linear(in_features=10, out_features=10, bias=True)
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): Linear(in_features=10, out_features=10, bias=True)
  (4): Linear(in_features=10, out_features=10, bias=Tru

In [32]:
# class torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)：对输入数据的最后一维进行卷积
# in_channels:输入信号的通道，可以理解为特征的维度
# out_channels:卷积产生的通道，可以理解为卷积核的数量
# kernel_size：卷积核的大小(只需要指定卷积核方向的大小)
# 一维卷积层，输入的尺度是(N, C_in,L)，输出尺度（ N,C_out,L_out）.
# 卷积的方向是一维的
import torch
conv1 = nn.Conv1d(256,100,2)
input = torch.randn(32,35,256)
input = input.permute(0,2,1) # 对最后一维进行卷积，设置卷积的方向正确
output = conv1(input)
print(output.size())

torch.Size([32, 1, 34])


In [ ]:
# class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
# in_channels:输入信号的通道
# out_channels:卷积产生的通道
# kernel_size：卷积核的尺寸
conv2 = nn.Conv2d(1,6,5)
# 例子：一个样本：32*32*1，卷积核：5*5*6，输出：28*28*6